# Data exploration

## Setting up

In [1]:
# run script that installs missing libraries
! chmod 755 scripts.sh
! ./scripts.sh

### Imports

In [1]:
import os
from matplotlib import pyplot as plt
import matplotlib.path as mpath
import numpy as np
import pandas as pd
import xarray as xr
import cartopy
import cf_units
from datetime import datetime
from datetime import timedelta
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

In [2]:
! ls ../data

MARcst-AN35km-176x148.cdf	      year_ACCESS1-3.nc2
year-MAR_ACCESS1.3-1980-2100_zen.nc2


## Global data:
See that global data has weird time format that needs to be fixed, format is number of months since starting date. Months are counted in 30 days per month, CF format.

In [3]:
dg = xr.open_dataset('../data/year_ACCESS1-3.nc2', decode_times=False)
dg.load()

<xarray.Dataset>
Dimensions:         (TIME2: 151, bnds: 2, LON: 192, LAT: 144, PLEV12_15: 4)
Coordinates:
  * TIME2           (TIME2) float64 5.5 17.5 29.5 ... 1.794e+03 1.806e+03
  * LON             (LON) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
  * LAT             (LAT) float64 -89.38 -88.12 -86.88 ... 86.88 88.12 89.38
  * PLEV12_15       (PLEV12_15) float64 5e+04 6e+04 7e+04 8.5e+04
Dimensions without coordinates: bnds
Data variables:
    TIME2_bnds      (TIME2, bnds) float64 0.0 11.0 12.0 ... 1.8e+03 1.811e+03
    PLEV12_15_bnds  (PLEV12_15, bnds) float64 4.5e+04 5.5e+04 ... 8.875e+04
    TA              (TIME2, PLEV12_15, LAT, LON) float32 232.7 232.7 ... 267.7
    TAS             (TIME2, LAT, LON) float32 225.6 225.6 225.5 ... 271.3 271.3
    UA              (TIME2, PLEV12_15, LAT, LON) float32 0.6384 ... -2.163
    VA              (TIME2, PLEV12_15, LAT, LON) float32 -1.574 ... -0.3436
    ZG              (TIME2, PLEV12_15, LAT, LON) float32 4.961e+03 ... 1.368e+03
Attributes:
    CDI:                       Climate Data Interface version 1.9.7.1 (http:/...
    Conventions:               CF-1.6
    history:                   Fri May 15 15:08:09 2020: cdo yearmean ACCESS1...
    NCO:                       netCDF Operators version 4.7.5 (Homepage = htt...
    nco_openmp_thread_number:  1
    frequency:                 year
    CDO:                       Climate Data Operators version 1.9.7.1 (http:/...

### Fix time format

In [4]:
# get starting date and units of dataset
units, reference_date = dg.TIME2.attrs['units'].split('since')
print('Units: {} and reference date: {} of global dataset'.format(units, reference_date))

# set reference date, add first value in time array
START = datetime(1950,1,15,0,0,0)+timedelta(days=dg.TIME2[0].values*30)

print('First date in time array: {}'.format(START))

# Correct the time by having a yearly range from the starting date till 2100
dg['TIME'] = pd.date_range(start=START, periods=dg.sizes['TIME2'], freq='12M')
dg.load()

Units: month(360days)  and reference date:  1950-01-15 00:00:00 of global dataset
First date in time array: 1950-06-29 00:00:00


<xarray.Dataset>
Dimensions:         (TIME2: 151, bnds: 2, LON: 192, LAT: 144, PLEV12_15: 4, TIME: 151)
Coordinates:
  * TIME2           (TIME2) float64 5.5 17.5 29.5 ... 1.794e+03 1.806e+03
  * LON             (LON) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
  * LAT             (LAT) float64 -89.38 -88.12 -86.88 ... 86.88 88.12 89.38
  * PLEV12_15       (PLEV12_15) float64 5e+04 6e+04 7e+04 8.5e+04
  * TIME            (TIME) datetime64[ns] 1950-06-30 1951-06-30 ... 2100-06-30
Dimensions without coordinates: bnds
Data variables:
    TIME2_bnds      (TIME2, bnds) float64 0.0 11.0 12.0 ... 1.8e+03 1.811e+03
    PLEV12_15_bnds  (PLEV12_15, bnds) float64 4.5e+04 5.5e+04 ... 8.875e+04
    TA              (TIME2, PLEV12_15, LAT, LON) float32 232.7 232.7 ... 267.7
    TAS             (TIME2, LAT, LON) float32 225.6 225.6 225.5 ... 271.3 271.3
    UA              (TIME2, PLEV12_15, LAT, LON) float32 0.6384 ... -2.163
    VA              (TIME2, PLEV12_15, LAT, LON) float32 -1.574 ... -0.3436
    ZG              (TIME2, PLEV12_15, LAT, LON) float32 4.961e+03 ... 1.368e+03
Attributes:
    CDI:                       Climate Data Interface version 1.9.7.1 (http:/...
    Conventions:               CF-1.6
    history:                   Fri May 15 15:08:09 2020: cdo yearmean ACCESS1...
    NCO:                       netCDF Operators version 4.7.5 (Homepage = htt...
    nco_openmp_thread_number:  1
    frequency:                 year
    CDO:                       Climate Data Operators version 1.9.7.1 (http:/...

### Some plots:

In [ ]:
decades = dg.resample(TIME="Y").mean().groupby("TIME.year").mean()
decades.TA.plot(col="year", row = "PLEV12_15")

In [ ]:
# mean SMB values over first 5 years
yearly_mean = dg.groupby("TIME.year").mean().isel(year=slice(5))
# facet the yearly means
yearly_mean.TA.plot(col="year", row = "PLEV12_15")

In [ ]:
# mean SMB values over first 5 years
yearly_mean = dg.groupby("TIME.year").mean().isel(year=slice(5))
# facet the yearly means
yearly_mean.TAS.plot(col="year")

## Exploring regional data:

In [ ]:
dsr = xr.open_dataset('../data/year-MAR_ACCESS1.3-1980-2100_zen.nc2')
ds = dsr.load()
ds

In [ ]:
# plot the first timestep
ds.SMB.isel(TIME=1).plot(x="X")

In [ ]:
# mean SMB over time
ds.SMB.mean("TIME").plot(x="X")
plt.title('Mean SMB values over time')

In [ ]:
# mean SF over time
ds.RF.mean("TIME").plot(x="X")
plt.title('Mean RF values over time')

In [ ]:
# mean RU over time
ds.RU.mean("TIME").plot(x="X")
plt.title('Mean RU values over time')

In [ ]:
# mean SU over time
ds.SU.mean("TIME").plot(x="X")
plt.title('Mean SU values over time')

In [ ]:
# mean SMB values over first 5 years
yearly_mean = ds.groupby("TIME.year").mean().isel(year=slice(5))
# facet the yearly means
yearly_mean.SMB.plot(col="year")

In [ ]:
decades = ds.resample(TIME="50Y").mean().groupby("TIME.year").mean()
decades.SF.plot(col="year")

In [ ]:
decades.SMB.plot(col="year")

---

## Extracting data or "indexing" : `.sel`, `.isel`

Xarray supports

- label-based indexing using `.sel`
- position-based indexing using `.isel`

For more see https://xarray.pydata.org/en/stable/indexing.html

### Label-based indexing

Xarray inherits its label-based indexing rules from pandas; this means great
support for dates and times!

In [ ]:
# pull out data for all of 2037-July
ds.sel(TIME="2037-07")

In [ ]:
# demonstrate slicing
ds.sel(TIME=slice("2013-05", "2013-07"))

In [ ]:
# demonstrate "nearest" indexing
ds.sel(X=240.2, method="nearest")

In [ ]:
# "nearest indexing at multiple points"
ds.sel(X=[240.125, 234], Y=[40.3, 50.3], method="nearest")

### Position-based indexing

This is similar to your usual numpy `array[0, 2, 3]` but with the power of named
dimensions!

In [ ]:
# pull out time index 0 and lat index 0
ds.SMB.isel(TIME=0, X=30)  #  much better than ds.air[0, 0, :]

In [ ]:
# demonstrate slicing
ds.SMB.isel(X=slice(10))

---

## High level computation: `groupby`, `resample`, `rolling`, `coarsen`, `weighted`

Xarray has some very useful high level objects that let you do common
computations:

1. `groupby` :
   [Bin data in to groups and reduce](https://xarray.pydata.org/en/stable/groupby.html)
1. `resample` :
   [Groupby specialized for time axes. Either downsample or upsample your data.](https://xarray.pydata.org/en/stable/time-series.html#resampling-and-grouped-operations)
1. `rolling` :
   [Operate on rolling windows of your data e.g. running mean](https://xarray.pydata.org/en/stable/computation.html#rolling-window-operations)
1. `coarsen` :
   [Downsample your data](https://xarray.pydata.org/en/stable/computation.html#coarsen-large-arrays)
1. `weighted` :
   [Weight your data before reducing](https://xarray.pydata.org/en/stable/computation.html#weighted-array-reductions)


### groupby

In [ ]:
# seasonal groups, see that we only have summer data (june, july, august)
ds.groupby("TIME.season")

In [ ]:
# make a seasonal mean
seasonal_mean = ds.groupby("TIME.season").mean()
seasonal_mean

### resample

In [ ]:
# resample to monthly frequency
ds.resample(TIME="M").mean()

---

## Visualization: `.plot`

For more see https://xarray.pydata.org/en/stable/plotting.html and
https://xarray.pydata.org/en/stable/examples/visualization_gallery.html

We have seen very simple plots earlier. Xarray has some support for visualizing
3D and 4D datasets by presenting multiple facets (or panels or subplots) showing
variations across rows and/or columns.


In [ ]:
# facet the seasonal_mean
seasonal_mean.SMB.plot()

In [ ]:
# contours
seasonal_mean.SMB.isel(SECTOR1_1=0, season =0).plot.contour(levels=800, add_colorbar=True)

In [ ]:
# line plots too? wut
seasonal_mean.SMB.isel(SECTOR1_1=0).mean("X").plot.line(hue="season", y="Y")

In [ ]:
import plotly.express as px

df = pd.DataFrame(data = {"time":ds.TIME.data, "data" : ds.SMB.data}) 

fig = px.line(df, x="time", y="data", title='Life expectancy in Canada')
fig.show()

In [ ]:
# pull out data for all of 2013-May
ds.sel(TIME="2037-07")